In [218]:
import pandas as pd
import numpy as np
idx = pd.IndexSlice
% matplotlib inline 
from util import error_function, get_simple_pred, get_quantil_pred
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [500]:
train = pd.read_csv('data/train.csv',parse_dates=['Date'])
test = pd.read_csv('data/test.csv',parse_dates=['Date'])

train = train[train.Date.dt.month==5]
train = train.set_index(['Date','City'])
train = train.sort_index()

train['day_of_week'] = train.index.get_level_values('Date').weekday_name

last_day_in_index = train.index.get_level_values('Date').max()
new_last_day_train = last_day_in_index - pd.DateOffset(days=7)
new_first_day_test = last_day_in_index - pd.DateOffset(days=6)
new_last_day_test = last_day_in_index - pd.DateOffset(days=3)
new_train = train.loc[idx[:new_last_day_train, :], :]
new_test = train.loc[idx[new_first_day_test:new_last_day_test, :], :]
new_test = new_test.reset_index()

In [501]:
new_train = new_train[new_train['Hospital active']==1]

In [502]:
df_quantil_Sun = new_train.groupby(['City','day_of_week'])['Nr_patients'].quantile(q=0.94)

In [503]:
df_quantil_Mon = new_train.groupby(['City','day_of_week'])['Nr_patients'].quantile(q=0.67)

In [522]:
df_quantil_Tue = new_train.groupby(['City','day_of_week'])['Nr_patients'].quantile(q=0.94)

In [523]:
df_quantil_Wen = new_train.groupby(['City','day_of_week'])['Nr_patients'].quantile(q=0.94)

In [524]:
df_mean = new_train.groupby('City')['Nr_patients'].max()
df_mean = pd.DataFrame(df_mean)

In [525]:
new_test['nr_doctors'] = new_test.apply(get_quantil_pred, axis=1,args=(df_quantil_Sun,df_quantil_Mon,df_quantil_Tue,df_quantil_Wen))

In [526]:
print(error_function(new_test['Nr_patients'],new_test['nr_doctors']))

1515.0


In [527]:
new_test.groupby('Date').sum()

,Doctors allocated,Nr_patients,Deaths from lack of doctors,National Holiday,News on local media about deseases,Students on Holiday,Hospital active,nr_doctors
Date,,,,,,,,
2018-05-13,1749,9549,0.0,0,0,0,12,750
2018-05-14,39665,230834,0.0,0,0,9,307,19915
2018-05-15,39665,225278,0.0,0,0,9,307,19928
2018-05-16,39665,265665,0.0,0,0,9,307,20073


In [496]:
test['nr_doctors'] = test.apply(get_quantil_pred, axis=1,args=(df_quantil_Sun,df_quantil_Mon,df_quantil_Tue,df_quantil_Wen))

In [497]:
test.groupby('Date').sum()

,Hospital active,nr_doctors
Date,,
2018-05-20,12,945
2018-05-21,308,19989
2018-05-22,308,19875
2018-05-23,308,19899


In [498]:
test.drop(columns=['Hospital active']).to_csv('submission2.csv',index=False)

In [351]:
df = pd.read_csv('submission1.csv')